# **StereoCamera cone detection**

## **Setup**

Setup the repo and the model.

### **Clone the repositories and import the libraries**

Clone the repository from the smart application course organization and the one for the yolov5 model and import the libraries needed to work with the porject.

In [1]:
!git clone --recurse-submodules -b dev_Petix_Ristori "https://github.com/unipi-smartapp-2021/lidar-cone-detection"   # clone our repository
!git clone "https://github.com/ultralytics/yolov5"  # clone the model repository

fatal: destination path 'lidar-cone-detection' already exists and is not an empty directory.
fatal: destination path 'yolov5' already exists and is not an empty directory.


Install the requirements.

In [ ]:
!pip install -r lidar-cone-detection/requirements.txt

Import the libraries and mount the drive.

In [2]:
import cv2
from tqdm import tqdm
import numpy as np
import py7zr
import zipfile
from google.colab import drive
drive.mount('/content/drive')  # mount the drive

Mounted at /content/drive


### **Manage the dataset**

There are two datasets, choose wisely:  
- the first dataset is the small one and contains 101 images.
- the second one is the full one with more than 8000 images.
- there will be more datasets in the future.

In [3]:
def extract7z(zip_path):
    with py7zr.SevenZipFile(zip_path, mode='r') as z:
        z.extractall()

def extractzip(zip_path):
    with zipfile.ZipFile(zip_path, mode='r') as z:
        z.extractall()

In [4]:
datasets = ["/content/drive/Shareddrives/Sensory_data/ConeDataset.7z",
            "/content/drive/Shareddrives/Sensory_data/TRset-20211117T155203Z-001.zip"]
datasets_extract = [extract7z, extractzip]

In [5]:
datasets_extract[0](datasets[0])

**Extract the chosen dataset**

Manage the dataset by following what is suggested in https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data chapter 1.3. Keep in mind to modify the directory from which you're moving the images (ConeDataset for the first dataset or TRset for the full one).

You can decide to convert the images to grayscale ones or not, then the dataset is splited into train, validation and test sets.

In [6]:
!python lidar-cone-detection/src/manage_dataset.py --dataset ConeDataset --path lidar-cone-detection/src/datasets/ConeDataset

### **Check the yolov5 model and import the weights**

Check if yolov5 is ready and fine.

In [7]:
%cd yolov5
from yolov5.utils import notebook_init
display = notebook_init()  # checks
%cd ../

YOLOv5 🚀 v6.0-120-g92a7391 torch 1.10.0+cu111 CPU


Setup complete ✅
/content


Import the yolov5 model, all the available models can be found here: https://github.com/ultralytics/yolov5/releases/.

In [8]:
import requests
url = 'https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt'
r = requests.get(url, allow_redirects=True)
open('yolov5s.pt', 'wb').write(r.content)

14698491

## **Train the model**

Train the model following the guide from https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb.

**Main arguments:**
*   *--img*, image size.
*   *--batch*, batch size.
*   *--epochs*, number of epochs.
*   *--data*, the yaml file that specifies where datasets are.
*   *--weights*, the model weights (you can find them here https://github.com/ultralytics/yolov5/releases).
*   *--cache*, to save a cache file of the train and validation sets.
*   *--project*, where to save the results.

**An example of train call would be:**

!python yolov5/train.py --img img_size --batch batch_size --epochs epochs --data dataset.yaml --weights model --cache --project runs/train








In [ ]:
!python yolov5/train.py --img 640 --batch 8 --epochs 100 --data lidar-cone-detection/src/stereocamera/conedataset.yaml --weights yolov5s.pt --cache --project runs/train

Use the tensorboard to see the results.

In [ ]:
# Tensorboard  (optional)
%reload_ext tensorboard
#%load_ext tensorboard
%tensorboard --logdir runs/train

## **Evaluate the model**

Use val.py if you have a test set defined in conedataset.yaml. Remember to use --task test or the model will work on the validation set.

In [ ]:
!python yolov5/val.py --weights /content/runs/train/exp2/weights/best.pt --img 640 --conf 0.5 --data lidar-cone-detection/src/stereocamera/conedataset.yaml --task test --project runs/test

val: data=lidar-cone-detection/src/stereocamera/conedataset.yaml, weights=['/content/drive/Shareddrives/Sensory_data/best.pt'], batch_size=32, imgsz=640, conf_thres=0.5, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/test, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-115-gbc48457 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning 'lidar-cone-detection/src/stereocamera/datasets/ConeDataset/test' images and labels...11 found, 0 missing, 0 empty, 0 corrupted: 100% 11/11 [00:00<00:00, 469.75it/s]
test: New cache created: lidar-cone-detection/src/stereocamera/datasets/ConeDataset/test.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 1/1 [00:01<00:00,  1.50s/it]
                 all         11        127     

## **Inference**

Use detect.py if you have no labeled images or you want to try the model on your custom data.

In [10]:
!rm -rf detect/exp
!python yolov5/detect.py --weights drive/Shareddrives/Sensory_data/best.pt --img 640 --conf 0.5 --source lidar-cone-detection/src/datasets/ConeDataset/images/image348.jpg --exist-ok --nosave --save-txt --project detect --hide-labels  --hide-conf

detect: weights=['drive/Shareddrives/Sensory_data/best.pt'], source=lidar-cone-detection/src/datasets/ConeDataset/images/image348.jpg, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=True, hide_conf=True, half=False, dnn=False
YOLOv5 🚀 v6.0-120-g92a7391 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/lidar-cone-detection/src/datasets/ConeDataset/images/image348.jpg: 384x640 5 oranges, 4 yellows, Done. (0.219s)
Speed: 1.3ms pre-process, 218.8ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to detect/exp
1 labels saved to detect/exp/labels
